In [26]:
import re

In [27]:
# Custom exception class for syntax errors
class SyntaxError(Exception):
    pass

In [28]:
# Function to tokenize the input expression
def tokenize(expression):
    tokens = []
    # Define token patterns using regular expressions
    patterns = [
        (r'[0-9]+', 'NUMBER'),
        (r'[a-zA-Z]+', 'VARIABLE'),
        (r'\+', 'ADD'),
        (r'-', 'SUB'),
        (r'\*', 'MUL'),
        (r'\(', 'LPAREN'),
        (r'\)', 'RPAREN')
    ]
    # Combine patterns into a single regular expression
    combined_patterns = '|'.join('(%s)' % pattern for pattern, _ in patterns)
    # Iterate over matches in the input expression
    for match in re.finditer(combined_patterns, expression):
        # Identify the matched token type and value
        for i, (pattern, token_type) in enumerate(patterns):
            if match.group(i + 1):
                tokens.append((token_type, match.group())) # Add token to the list
    return tokens

In [29]:
# Function to parse an expression
def parse_expr(tokens):
    term_node = parse_term(tokens) # Parse the term
    expr_prime_node = parse_expr_prime(tokens) # Parse the expression prime
    if tokens:
        raise SyntaxError("Unexpected tokens at the end of the expression") # Check for unexpected tokens
    return ('expr', term_node, expr_prime_node) # Return the parse tree for the expression

# Function to parse expression primes (handles addition and subtraction)
def parse_expr_prime(tokens):
    if tokens and tokens[0][1] in ('+', '-'): # Check for addition or subtraction operator
        operator = tokens[0][1] # Get the operator
        tokens.pop(0)  # Consume the operator token
        term_node = parse_term(tokens) # Parse the term
        expr_prime_node = parse_expr_prime(tokens) # Parse the expression prime recursively
        return ('expr_prime', operator, term_node, expr_prime_node) # Return the parse tree node for expression prime
    else:
        return ('expr_prime', None) # Return None if no expression prime

# Function to parse a term
def parse_term(tokens):
    factor_node = parse_factor(tokens) # Parse the factor
    term_prime_node = parse_term_prime(tokens) # Parse the term prime
    return ('term', factor_node, term_prime_node) # Return the parse tree node for the term

# Function to parse term primes (handles multiplication)
def parse_term_prime(tokens):
    if tokens and tokens[0][1] == '*': # Check for multiplication operator
        tokens.pop(0)  # Consume the '*' token
        factor_node = parse_factor(tokens) # Parse the factor
        term_prime_node = parse_term_prime(tokens) # Parse the term prime recursively
        return ('term_prime', '*', factor_node, term_prime_node) # Return the parse tree node for term prime
    else:
        return ('term_prime', None) # Return None if no term prime

# Function to parse a factor
def parse_factor(tokens):
    token_type, token_value = tokens.pop(0) # Get the token type and value
    if token_type == 'NUMBER' or token_type == 'VARIABLE': # Check if token is a number or variable
        return ('factor', token_type, token_value) # Return the parse tree node for factor
    elif token_type == 'LPAREN': # Check if token is a left parenthesis
        expr_node = parse_expr(tokens)  # Parse the enclosed expression
        if tokens[0][0] != 'RPAREN': # Check for matching right parenthesis
            raise SyntaxError("Missing closing parenthesis") # Raise error if closing parenthesis is missing
        tokens.pop(0)  # Consume the ')' token
        return ('factor', 'EXPR', expr_node) # Return the parse tree node for factor with enclosed expression
    else:
        raise SyntaxError("Unexpected token: %s" % token_value) # Raise error for unexpected token

# Function to test the parser with given expressions   
def test_parser(expressions):
    for expr in expressions:
        print("Parsing:", expr) # Print the expression being parsed
        try:
            tokens = tokenize(expr) # Tokenize the expression
            parse_tree = parse_expr(tokens) # Parse the expression
            print("Parse Tree:", parse_tree) # Print the parse tree
        except SyntaxError as e:
            print("Syntax Error:", e) # Handle syntax errors
        print() # Print newline for clarity

In [30]:
def main():
    expressions = [
        "a + 5",
        "3 * b - 2",
        "x * (y + 7)",
        "a / 5",
        "3 * [ b - 2]",
        "x / {y + 7}"
    ]
    test_parser(expressions)

if __name__ == "__main__":
    main()

Parsing: a + 5
Parse Tree: ('expr', ('term', ('factor', 'VARIABLE', 'a'), ('term_prime', None)), ('expr_prime', '+', ('term', ('factor', 'NUMBER', '5'), ('term_prime', None)), ('expr_prime', None)))

Parsing: 3 * b - 2
Parse Tree: ('expr', ('term', ('factor', 'NUMBER', '3'), ('term_prime', '*', ('factor', 'VARIABLE', 'b'), ('term_prime', None))), ('expr_prime', '-', ('term', ('factor', 'NUMBER', '2'), ('term_prime', None)), ('expr_prime', None)))

Parsing: x * (y + 7)
Syntax Error: Unexpected tokens at the end of the expression

Parsing: a / 5
Syntax Error: Unexpected tokens at the end of the expression

Parsing: 3 * [ b - 2]
Parse Tree: ('expr', ('term', ('factor', 'NUMBER', '3'), ('term_prime', '*', ('factor', 'VARIABLE', 'b'), ('term_prime', None))), ('expr_prime', '-', ('term', ('factor', 'NUMBER', '2'), ('term_prime', None)), ('expr_prime', None)))

Parsing: x / {y + 7}
Syntax Error: Unexpected tokens at the end of the expression

